In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
import time

In [4]:
DATA_PATH = os.path.join("DATA_TEST") 
actions = np.array(["HANDCLAPPING"])
no_sequences = 1775
sequence_length = 30

In [5]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [6]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 
pTime = 0
cTime = 0

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                 
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (700,image.shape[0]))
    return image, results
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,225,10), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(0,34,121), thickness=2, circle_radius=2)
                             ) 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([pose])

In [8]:
path = "C:/Users/Nguyễn Hoài Nam/OneDrive/CDIO4/stand_video"
video_list = os.listdir(path)
for video in video_list:
    cd = os.path.join(path +"/"+ video)
    cap = cv2.VideoCapture(cd)
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for action in actions:
            for sequence in range(1270,no_sequences):
                for frame_num in range(sequence_length):
                        ref,frame = cap.read()
                        try:
                            image, results = mediapipe_detection(frame, holistic)
                        except:
                            break
                            
                        draw_styled_landmarks(image, results)
                        cTime = time.time()
                        fps = 1 / (cTime - pTime)
                        pTime = cTime
                        cv2.putText(image,"FPS:" +str(int(fps)),(10,100), cv2.FONT_HERSHEY_PLAIN, 2,(255,0,190),2,cv2.LINE_AA)

                        if frame_num == 0: 
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)
                        else: 
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)

                        keypoints = extract_keypoints(results)
                        npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                        np.save(npy_path, keypoints)
                        if cv2.waitKey(1) & 0xFF == ord("q"):
                            break
cap.release()
cv2.destroyAllWindows()
cap.release()
cv2.destroyAllWindows()

## Get Labels

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [5]:
DATA_PATH = os.path.join('data_npy') 
actions = np.array(["HANDCLAPPING","RUNNING","WALKING","STANDING"])
no_sequences = 1776
sequence_length = 30

In [6]:
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'HANDCLAPPING': 0, 'RUNNING': 1, 'WALKING': 2, 'STANDING': 3}

In [15]:
def load_data(actions,no_sequences,sequence_length):
    sequences1,labels = [],[]
    for action in actions:
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
                window.append(res)
            sequences1.append(window)
            labels.append(label_map[action])
    return sequences1,labels

In [7]:
# sequences,labels = [],[]
# for action in os.listdir("xxx/"):
#     fd = os.path.join("xxx/" + action)
#     window = []
#     for folder in os.listdir(fd):
#         fd1 = os.path.join("xxx/" + action +"/"+ folder)
#         for file in os.listdir(fd1):
#             res = np.load(os.path.join("xxx/" + action +"/"+ folder + "/" + file))
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[action])

In [19]:
%time sequences,labels = load_data(actions,no_sequences,sequence_length)

Wall time: 40min 17s


In [25]:
X = np.array(sequences)
X.shape

(7104, 30, 132)

In [26]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [27]:
with open('X1.npy', 'wb') as f:
    np.save(f, X)
with open('y1.npy', 'wb') as f:
    np.save(f, y)